In [1]:
#ProgramName:     Daily-UpdateRowCountDifferenceValues-DEV
#Purpose:         Update default values(zero) differences with calculated changes captured
#Author:          Greg Turmel, Director, Data Governance 
#Date:            2020.08.30 - 2021.06.30
#Errata:          0.1 Improvements can be made to script using for/looping through the metadata captured

import os, sys, argparse, csv, pyodbc, sql, time, datetime
import sqlalchemy as db
import errno, pathlib2

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [2]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running
# ===================================================================================================================
pn = r'eeeeeeeeee'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
an = r'Difference\Process'
bn = r'Difference\Reports'

sn = r'Difference\sql'
tn = r'config'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathDP = os.path.join(parentDirectory, programDirectory, an)
pathDR = os.path.join(parentDirectory, programDirectory, bn)
pathDS = os.path.join(parentDirectory, programDirectory, sn)
pathDT = os.path.join(parentDirectory, programDirectory, tn)

procpath = pathDP
csvpath = pathDR
sqlpath = pathDS
configpath = pathDT

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathDP):
        os.makedirs(pathDP, mode)
        print('Program Directory subfolder has been created: ', pathDP)
    else:
        print('Directory:', pathDP, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathDR):
        os.makedirs(pathDR, mode)
        print('Program Directory subfolder has been created: ', pathDR)
    else:
        print('Directory:', pathDR, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathDS):
        os.makedirs(pathDS, mode)
        print('Program Directory subfolder has been created: ', pathDS)
    else:
        print('Directory:', pathDS, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathDT):
        os.makedirs(pathDT, mode)
        print('Program Directory subfolder has been created: ', pathDT)
    else:
        print('Central Config Directory:', pathDT, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

# SQL files for reset first value in diff column to ensure calculations are not skewed
# Found in C:\Users\eeeeeeeeeeee\Documents\Py\Daily\Difference\sql
#ApplicationData
diffAD = 'SQLQuery-updateAppDatadifferencevaluesRow1diffeq0.sql'
#Assessment
diffAM = 'SQLQuery-updateAssessmentdifferencevaluesRow1diffeq0.sql'
#Control
diffCL = 'SQLQuery-updateControldifferencevaluesRow1diffeq0.sql'
#DownstreamFeeds
diffDF = 'SQLQuery-updateDownstreamFeedsdifferencevaluesRow1diffeq0.sql'
#DownstreamFeeds_V1.5
diffDV = 'SQLQuery-updateDownstreamFeedsV15differencevaluesRow1diffeq0.sql'
#GCPS_Operations
diffGO = 'SQLQuery-updateGCPS_OperationsdifferencevaluesRow1diffeq0.sql'
#GSDR
diffGR = 'SQLQuery-updateGSDRdifferencevaluesRow1diffeq0.sql'
#GSDR_Synergy
diffGS = 'SQLQuery-updateGSDR_SynergydifferencevaluesRow1diffeq0.sql'
#GSDR_Synergy_Temp
diffGP = 'SQLQuery-updateGSDR_Synergy_TempdifferencevaluesRow1diffeq0.sql'
#GSDR_Temp
diffGT = 'SQLQuery-updateGSDR_TempdifferencevaluesRow1diffeq0.sql'
#ODS_WebApps
diffOW = 'SQLQuery-updateODS_WebAppsdifferencevaluesRow1diffeq.sql'
#PPROD
diffPP = 'SQLQuery-updatePPRODdifferencevaluesRow1diffeq0.sql'
#Predictive_Analytics
diffPV = 'SQLQuery-updatePredictive_AnalyticsdifferencevaluesRow1diffeq0.sql'


Parent Directory is:  C:\Users\e201873842\Documents\Py
Directory: Daily >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Difference\Process >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Difference\Reports >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Difference\sql >>>>> Note ---- this folder already exists <<<<<
Central Config Directory: C:\Users\e201873842\Documents\Py\Daily\config >>>>> Note ---- this folder already exists <<<<<


In [3]:
conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
sql_query = pd.read_sql_query(''' 
SELECT DISTINCT [TableName] FROM [Greg].[dbo].[tableRowCountApplicationData] ORDER BY 1;
'''
                              ,conn)
conn.close()

In [4]:
#Read into the df the sql file built as a CTE update command
now01 = datetime.datetime.now()
print('1.1.0 Begin AppData Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

s01 = open(os.path.join(pathDS, 'update_appdata_set_difference.sql'))
sqlFile01 = s01.read()
s01.close()

for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.0 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.0 Begin AppData Difference Value update:  2021-06-14 07:26:46
1.1.0 Difference Value updated - Process Complete:  2021-06-14 07:27:15


In [5]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#ApplicationData reset

now01 = datetime.datetime.now()
print('1.1.1 Begin AppData Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffAD))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.1 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.1 Begin AppData Row 1 = 0 Difference Value update:  2021-06-14 07:27:15
1.1.1 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:27:19


In [6]:
s02 = open(os.path.join(pathDS, 'update_assessment_set_difference.sql'))
sqlFile02 = s02.read()
s02.close()

for sqlCmd in sqlFile02.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.2 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.2 Difference Value updated - Process Complete:  2021-06-14 07:30:48


In [7]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#Assessment reset

now01 = datetime.datetime.now()
print('1.1.3 Begin Assessment Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffAM))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.3 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.3 Begin Assessment Row 1 = 0 Difference Value update:  2021-06-14 07:30:48
1.1.3 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:31:07


In [8]:
s03 = open(os.path.join(pathDS, 'update_control_set_difference.sql'))
sqlFile03 = s03.read()
s03.close()

for sqlCmd in sqlFile03.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.4 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.4 Difference Value updated - Process Complete:  2021-06-14 07:31:19


In [9]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#Control reset

now01 = datetime.datetime.now()
print('1.1.5 Begin Control Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffCL))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.5 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.5 Begin Control Row 1 = 0 Difference Value update:  2021-06-14 07:31:19
1.1.5 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:31:22


In [10]:
s04 = open(os.path.join(pathDS, 'update_downstreamfeeds_set_difference.sql'))
sqlFile04 = s04.read()
s04.close()

for sqlCmd in sqlFile04.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.6 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.6 Difference Value updated - Process Complete:  2021-06-14 07:34:19


In [11]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#DownstreamFeeds reset

now01 = datetime.datetime.now()
print('1.1.7 Begin DownstremFeeds Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffDF))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.7 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.7 Begin DownstremFeeds Row 1 = 0 Difference Value update:  2021-06-14 07:34:19
1.1.7 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:34:34


In [12]:
s05 = open(os.path.join(pathDS, 'update_downstreamfeedsV15_set_difference.sql'))
sqlFile05 = s05.read()
s05.close()

for sqlCmd in sqlFile05.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.8 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.8 Difference Value updated - Process Complete:  2021-06-14 07:34:41


In [13]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#DownstreamFeeds_V1.5 reset

now01 = datetime.datetime.now()
print('1.1.9 Begin DownstremFeeds_V1.5 Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffDV))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.9 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.9 Begin DownstremFeeds_V1.5 Row 1 = 0 Difference Value update:  2021-06-14 07:34:41
1.1.9 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:34:43


In [14]:
s06 = open(os.path.join(pathDS, 'update_gcps_operations_set_difference.sql'))
sqlFile06 = s06.read()
s06.close()

for sqlCmd in sqlFile06.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.10 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.10 Difference Value updated - Process Complete:  2021-06-14 07:34:51


In [15]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#GCPS_Operations reset

now01 = datetime.datetime.now()
print('1.1.11 Begin GCPS_Operations Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffGO))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.11 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.11 Begin GCPS_Operations Row 1 = 0 Difference Value update:  2021-06-14 07:34:51
1.1.11 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:34:53


In [16]:
s07 = open(os.path.join(pathDS, 'update_gsdr_set_difference.sql'))
sqlFile07 = s07.read()
s07.close()

for sqlCmd in sqlFile07.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.12 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.12 Difference Value updated - Process Complete:  2021-06-14 07:39:57


In [17]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#GSDR reset

now01 = datetime.datetime.now()
print('1.1.13 Begin GSDR Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffGR))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.13 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.13 Begin GSDR Row 1 = 0 Difference Value update:  2021-06-14 07:39:57
1.1.13 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:40:21


In [18]:
s08 = open(os.path.join(pathDS, 'update_gsdr_synergy_set_difference.sql'))
sqlFile08 = s08.read()
s08.close()

for sqlCmd in sqlFile08.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.14 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.14 Difference Value updated - Process Complete:  2021-06-14 07:40:27


In [19]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#GSDR_Synergy reset

now01 = datetime.datetime.now()
print('1.1.15 Begin GSDR_Synergy Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffGS))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.15 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.15 Begin GSDR_Synergy Row 1 = 0 Difference Value update:  2021-06-14 07:40:27
1.1.15 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:40:29


In [20]:
s09 = open(os.path.join(pathDS, 'update_gsdr_synergy_temp_set_difference.sql'))
sqlFile09 = s09.read()
s09.close()

for sqlCmd in sqlFile09.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.16 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.16 Difference Value updated - Process Complete:  2021-06-14 07:40:34


In [21]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#GSDR_Synergy_Temp reset

now01 = datetime.datetime.now()
print('1.1.17 Begin GSDR_Synergy_Temp Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffGP))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.17 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.17 Begin GSDR_Synergy_Temp Row 1 = 0 Difference Value update:  2021-06-14 07:40:34
1.1.17 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:40:37


In [22]:
s10 = open(os.path.join(pathDS, 'update_gsdr_temp_set_difference.sql'))
sqlFile10 = s10.read()
s10.close()

for sqlCmd in sqlFile10.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.18 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.18 Difference Value updated - Process Complete:  2021-06-14 07:45:23


In [23]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#GSDR_Temp reset

now01 = datetime.datetime.now()
print('1.1.19 Begin GSDR_Temp Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffGT))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.19 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.19 Begin GSDR_Temp Row 1 = 0 Difference Value update:  2021-06-14 07:45:23
1.1.19 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:45:46


In [24]:
s11 = open(os.path.join(pathDS, 'update_ods_webapps_set_difference.sql'))
sqlFile11 = s11.read()
s11.close()

for sqlCmd in sqlFile11.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.20 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.20 Difference Value updated - Process Complete:  2021-06-14 07:45:54


In [25]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#ODS_WebApps reset

now01 = datetime.datetime.now()
print('1.1.21 Begin ODS_WebApps Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffOW))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.21 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.21 Begin ODS_WebApps Row 1 = 0 Difference Value update:  2021-06-14 07:45:54
1.1.21 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:45:56


In [26]:
s12 = open(os.path.join(pathDS, 'update_pprod_set_difference.sql'))
sqlFile12 = s12.read()
s12.close()

for sqlCmd in sqlFile12.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.22 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.22 Difference Value updated - Process Complete:  2021-06-14 07:46:07


In [27]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#PPROD reset

now01 = datetime.datetime.now()
print('1.1.23 Begin PPROD Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffPP))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.23 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.23 Begin PPROD Row 1 = 0 Difference Value update:  2021-06-14 07:46:07
1.1.23 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:46:09


In [28]:
s13 = open(os.path.join(pathDS, 'update_predictive_analytics_set_difference.sql'))
sqlFile13 = s13.read()
s13.close()

for sqlCmd in sqlFile13.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.24 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.24 Difference Value updated - Process Complete:  2021-06-14 07:47:14


In [29]:
#Set the first row for each table to zero to prevent the avg and sum calculations in step 6 from being skewed
#Predictive_Analytics reset

now01 = datetime.datetime.now()
print('1.1.25 Begin Predictive_Analytics Row 1 = 0 Difference Value update: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read the sql file built as a CTE update command
s01 = open(os.path.join(pathDS, diffPV))
sqlFile01 = s01.read()
s01.close()

#strip and then split read on';' 
for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('1.1.25 Row 1 Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


1.1.25 Begin Predictive_Analytics Row 1 = 0 Difference Value update:  2021-06-14 07:47:14
1.1.25 Row 1 Difference Value updated - Process Complete:  2021-06-14 07:47:20


In [30]:
now01 = datetime.datetime.now()
print('Process Complete: All diff values updated: ', now01.strftime("%Y-%m-%d %H:%M:%S"))
print('Next, update Master and validate: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Process Complete: All diff values updated:  2021-06-14 07:47:20
Next, update Master and validate:  2021-06-14 07:47:20
